In [ ]:
import pandas as pd
import json
import os
import random

# Fichier Excel contenant les élèves
excel_file = "Classeur_étudiants.xlsx"  # Remplacer par le chemin de votre fichier Excel
sheet_name_03 = "G03"
history_file_03 = "group_history_03.json"

# Charger les étudiants depuis une feuille Excel
def load_students_from_excel(file_path, sheet_name):
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    if "NOM" not in df.columns or "PRÉNOM" not in df.columns:
        raise ValueError("Le fichier Excel doit contenir des colonnes 'NOM' et 'PRÉNOM'.")
    
    students = [f"{prenom} {nom}" for prenom, nom in zip(df["PRÉNOM"], df["NOM"])]
    return students

# Charger l'historique des groupes
def load_history(file_path):
    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)
    return []

# Sauvegarder l'historique
def save_history(history, file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(history, f, ensure_ascii=False, indent=4)

# Générer des groupes aléatoires uniques
def generate_groups(students, num_groups, history, max_attempts=100):
    attempts = 0
    while attempts < max_attempts:
        attempts += 1
        random.shuffle(students)
        groups = [students[i::num_groups] for i in range(num_groups)]

        if not any(groups == past for past in history):
            history.append(groups)
            return groups, history

    raise RuntimeError("Impossible de générer des groupes uniques après plusieurs essais.")

# Exécution principale
try:
    students = load_students_from_excel(excel_file, sheet_name_03)
    num_groups = 6
    history = load_history(history_file_03)
    
    groups, updated_history = generate_groups(students, num_groups, history)

    # Affichage des groupes
    for i, group in enumerate(groups, start=1):
        rapporteur = group[0] if group else "Aucun"
        print(f"Groupe {i} : {', '.join(group)} (Rapporteur : {rapporteur})")

    # Sauvegarde de l'historique mis à jour
    save_history(updated_history, history_file_03)

except Exception as e:
    print("Erreur :", str(e))
